# **Data Preprocessing**

In [1]:
# Importing the drive module from google.colab library
from google.colab import drive

# Mounting the Google Drive to the Colab environment
drive.mount('/content/drive')

project_path = '/content/drive/My Drive/GitHub/MarineMammalSoundClassification/'
%cd /content/drive/My Drive/GitHub/MarineMammalSoundClassification/

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1oJSL58N419Ve8pd0wCvgXEy52hLM2tJN/MarineMammalSoundClassification


In [5]:
import os
import shutil
import json
from utils.utilities import ensure_dir, get_wav_duration

## **Data Cleaning**

During the initial preprocessing, folders containing fewer than 20 instances were deleted. We then attempted to use the pyAudioAnalysis library to split the data into training, validation, and test sets. However, some files in the remaining folders produced errors, including an entire class (Fin_FinbackWhale). Consequently, during a second round of preprocessing, this class folder and the problematic files within other folders were also deleted.

Overall, the following folders of classes and specific files from the remaining classes have been removed from our data:

Directories:

*   **data/LeopardSeal** (10 instances)
*   **data/MinkeWhale** (17 instances)
*   **data/WeddellSeal** (2 instances)
*   **data/Fin_FinbackWhale** (IndexError: index 15 is out of bounds for axis 0 with size 15)

Files:

*   **data/PantropicalSpottedDolphin/84021003.wav** (WAV header is invalid: nAvgBytesPerSec must equal product of nSamplesPerSec and nBlockAlign, but file has nSamplesPerSec = 40960, nBlockAlign = 2, and nAvgBytesPerSec = 61440)
*   **data/Short_Finned(Pacific)PilotWhale/57021003.wav** (WAV header is invalid: nAvgBytesPerSec must equal product of nSamplesPerSec and nBlockAlign, but file has nSamplesPerSec = 30000, nBlockAlign = 2, and nAvgBytesPerSec = 45000)
*   **data/SpermWhale/84021003.wav** (WAV header is invalid: nAvgBytesPerSec must equal product of nSamplesPerSec and nBlockAlign, but file has nSamplesPerSec = 40960, nBlockAlign = 2, and nAvgBytesPerSec = 61440)


In [ ]:
directory_paths = ['data/LeopardSeal', 'data/MinkeWhale', 'data/WeddellSeal', 'data/Fin_FinbackWhale']

for dpath in directory_paths:
  if os.path.exists(dpath):
      shutil.rmtree(dpath)
      print(f"The directory {dpath} has been deleted.")
  else:
      print(f"The directory {dpath} does not exist.")

file_paths = ['data/PantropicalSpottedDolphin/84021003.wav', 'data/Short_Finned(Pacific)PilotWhale/57021003.wav', 'data/SpermWhale/84021003.wav']

for fpath in file_paths:
  if os.path.exists(fpath):
      os.remove(fpath)
      print(f"The file {fpath} has been deleted.")
  else:
      print(f"The file {fpath} does not exist.")

The directory data/LeopardSeal has been deleted.
The directory data/MinkeWhale has been deleted.
The directory data/WeddellSeal has been deleted.
The directory data/Fin_FinbackWhale has been deleted.
The file data/PantropicalSpottedDolphin/84021003.wav has been deleted.
The file data/Short_Finned(Pacific)PilotWhale/57021003.wav has been deleted.
The file data/SpermWhale/84021003.wav has been deleted.


We also delete any files that, according to their metadata, appear to contain <u>more than one mammal sounds</u>.

In [7]:
# Load metadata from JSON files
with open('metadata/wav_metadata.json', 'r') as f:
    wav_metadata = json.load(f)

with open('metadata/species.json', 'r') as f:
    species = json.load(f)

for m in wav_metadata:
    na = wav_metadata[m]['NA:'].replace("  ", " ")
    num_of_animals = na.split('|')

    if len(num_of_animals) > 1:
        print(m+'.wav')
        print(na)

        animal_1_code = num_of_animals[0].split(" ")[-2]
        animal_2_code = num_of_animals[1].split(" ")[-1]

        animal_1_name = next((s for s in species if species[s]['code'] == animal_1_code), None)
        animal_2_name = next((s for s in species if species[s]['code'] == animal_2_code), None)

        if animal_1_name and animal_2_name:
            print(animal_1_name, animal_2_name)

            filepath_1 = os.path.join('data', animal_1_name, m + '.wav')
            filepath_2 = os.path.join('data', animal_2_name, m + '.wav')

            if os.path.exists(filepath_1):
                os.remove(filepath_1)

            if os.path.exists(filepath_2):
                os.remove(filepath_2)

72021005.wav
1, possibly 2+ AA1A | 1+ CC2A
BowheadWhale BeardedSeal
7202100T.wav
1+ AA1A | 1+ CC2A
BowheadWhale BeardedSeal
7202100V.wav
1+ AA1A | 1+ CC2A
BowheadWhale BeardedSeal
7202100Z.wav
1+ AA1A | 1+ CC2A
BowheadWhale BeardedSeal
78018002.wav
3+ CC2A | 1+ AA1A
BeardedSeal BowheadWhale
78018003.wav
3+ CC2A | 1+ AA1A
BeardedSeal BowheadWhale
7801800B.wav
1+ AA1A | 3+ CC2A
BowheadWhale BeardedSeal
7801800D.wav
1+ AA1A | 3+ CC2A
BowheadWhale BeardedSeal
7801800H.wav
1+ AA1A | 3+ CC2A
BowheadWhale BeardedSeal
7801800J.wav
1+ AA1A | 3+ CC2A
BowheadWhale BeardedSeal
91012009.wav
2+ BD10A | 1+ BA2A
MelonHeadedWhale SpermWhale
9101200B.wav
2 BD10A | 1+ BA2A
MelonHeadedWhale SpermWhale
9101200K.wav
2+ BD10A | 1+ BA2A
MelonHeadedWhale SpermWhale
9101201E.wav
1 BD10A | 1 BA2A
MelonHeadedWhale SpermWhale
84021003.wav
1 BA2A | 3 BD15A
SpermWhale PantropicalSpottedDolphin
90058068.wav
3 BA2A | 100-150 BD15A
SpermWhale PantropicalSpottedDolphin
9005807N.wav
3 BA2A | 100-150 BD15A
SpermWhale Pant

## **Convert Sample Rate**

Our EDA revealed that the .wav files have varying sample rates across different species. Therefore, before creating spectrograms and testing the models, <u>we standardize all .wav files to a sample rate of 22050</u>, using the [ffmpeg-python](https://github.com/kkroening/ffmpeg-python) library.

In [8]:
!pip install ffmpeg-python

In [9]:
import ffmpeg

def convert_sample_rate(input_file, output_file, new_sample_rate):
    """
    Convert the sample rate of an audio file to a new specified rate.

    Parameters:
    input_file (str): The path to the input audio file.
    output_file (str): The path where the output audio file will be saved.
    new_sample_rate (int): The new sample rate to set for the audio file.

    Returns:
    None
    """
    (
        ffmpeg
        .input(input_file)
        .output(output_file, ar=new_sample_rate)
        .overwrite_output()
        .run()
    )

In [10]:
# Define the input (source) and output(destination) directories
## Source and destination directories
source_dir = 'data'
destination_dir = 'data_22050'

# Ensure the destination directory exists
ensure_dir(destination_dir)

## Walk through all files in the source directory
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if file.lower().endswith('.wav'):
            input_file = os.path.join(root, file)
            print('Convert: '+input_file)
            relative_path = os.path.relpath(input_file, source_dir)
            output_file = os.path.join(destination_dir, relative_path)
            ensure_dir(os.path.dirname(output_file))
            convert_sample_rate(input_file, output_file, 22050)

Convert: data/AtlanticSpottedDolphin/61025001.wav
Convert: data/AtlanticSpottedDolphin/61025002.wav
Convert: data/AtlanticSpottedDolphin/61025003.wav
Convert: data/AtlanticSpottedDolphin/61025004.wav
Convert: data/AtlanticSpottedDolphin/61025006.wav
Convert: data/AtlanticSpottedDolphin/61025007.wav
Convert: data/AtlanticSpottedDolphin/61025008.wav
Convert: data/AtlanticSpottedDolphin/61025009.wav
Convert: data/AtlanticSpottedDolphin/6102500A.wav
Convert: data/AtlanticSpottedDolphin/6102500B.wav
Convert: data/AtlanticSpottedDolphin/6102500C.wav
Convert: data/AtlanticSpottedDolphin/6102500D.wav
Convert: data/AtlanticSpottedDolphin/6102500E.wav
Convert: data/AtlanticSpottedDolphin/6102500F.wav
Convert: data/AtlanticSpottedDolphin/6102500G.wav
Convert: data/AtlanticSpottedDolphin/6102500H.wav
Convert: data/AtlanticSpottedDolphin/6102500I.wav
Convert: data/AtlanticSpottedDolphin/6102500J.wav
Convert: data/AtlanticSpottedDolphin/6102500K.wav
Convert: data/AtlanticSpottedDolphin/6102500M.wav


## **Data Splitting**

Next, the [split-folders](https://pypi.org/project/split-folders/) library was used to split folders with .wav files into train, validation and test (dataset) folders. The ratio we chose was 80% train, 10% validation, and 10% test because the dataset has a fairly small number of instances per class and many classes. We wanted to ensure that there would be a sufficient amount of data for training the model.

In [11]:
!pip install split-folders

Before splitting the files into train, validation, and test sets, we ensure that <u>all the long files (>30secs) are included in the train set</u> to prevent them from affecting the majority voting in the test set.

In [14]:
ensure_dir('data_split/train')

In [15]:
source_dir = 'data_22050'
destination_dir = 'data_split/train'

## Walk through all files in the source directory
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if file.lower().endswith('.wav'):
            input_file = os.path.join(root, file)
            duration = get_wav_duration(input_file)
            relative_path = os.path.relpath(input_file, source_dir)
            output_file = os.path.join(destination_dir, relative_path)
            ensure_dir(os.path.dirname(output_file))
            if duration > 30:
                shutil.move(input_file, output_file)

In [16]:
import splitfolders

# Split the dataset into train, validation, and test sets
# Parameters:
# - "data": The path to the original dataset directory
# - output="data_split": The path where the split data will be saved
# - seed=1337: Seed for random number generator to ensure reproducibility
# - ratio=(.8, .1, .1): The split ratio for train, validation, and test sets
# - group_prefix=None: Option to keep files with the same prefix together, set to None as it's not needed here
# - move=True: Move files instead of coping them
splitfolders.ratio("data_22050", output="data_split", seed=1337, ratio=(.8, .1, .1), group_prefix=None, move=True)

Copying files: 1527 files [00:09, 159.73 files/s]


In [19]:
# Delete the empty folder data_22050
shutil.rmtree('data_22050')